In [1]:
# Logger
import logging
logging.basicConfig(level=logging.WARN)

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load BACS module
from bacs.agents.bacs import BACS, Configuration
from bacs.agents.bacs.utils.GymMazeWrapper import _maze_metrics, parse_metrics_to_df, plot_performance

# Load environments
import gym
import my_mazes

# Allow to parallelize all benchmarks to do
import parmap

# For calculation standard deviation
import statistics

# Agent - BACS - BENCHMARKING

## Main Parameters

In [2]:
CLASSIFIER_LENGTH = 8
NUMBER_OF_POSSIBLE_ACTIONS = 8

NUMBER_OF_EXPLORE_TRIALS = 1000
DO_ACTION_PLANNING_EXPLORE = False
DO_GA_EXPLORE = False
DO_SUBSUMPTION_EXPLORE = True
BETA_EXPLORE = 0.05
EPSILON = 0.8

NUMBER_OF_EXPLOIT_TRIALS = 500
DO_ACTION_PLANNING_EXPLOIT = False
DO_SUBSUMPTION_EXPLOIT = True
BETA_EXPLOIT = 0.00

LENGTH_OF_BEHAVIORAL_SEQUENCES = 2

NUMBER_OF_ITERATIONS_TO_BENCH = 30

## Full Details of Agent Parameters

In [3]:
cfg_explore = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=DO_GA_EXPLORE,
    do_subsumption=DO_SUBSUMPTION_EXPLORE,
    do_action_planning=DO_ACTION_PLANNING_EXPLORE,
    action_planning_frequency=50,
    beta=BETA_EXPLORE,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=EPSILON,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

cfg_exploit = Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=NUMBER_OF_POSSIBLE_ACTIONS,
    user_metrics_collector_fcn=_maze_metrics,
    metrics_trial_frequency=1,
    do_ga=False,
    do_subsumption=DO_SUBSUMPTION_EXPLOIT,
    do_action_planning=DO_ACTION_PLANNING_EXPLOIT,
    action_planning_frequency=50,
    beta=BETA_EXPLOIT,
    gamma=0.95,
    theta_i=0.1,
    theta_r=0.9,
    epsilon=0.0,
    u_max=CLASSIFIER_LENGTH,
    theta_exp=20,
    theta_ga=100,
    theta_as=20,
    mu=0.3,
    chi=0.8,
    bs_max=LENGTH_OF_BEHAVIORAL_SEQUENCES
)

### Benchmarking - Maze

In [4]:
def bench_on_maze(env,n):
    avg_explore_list = []
    avg_exploit_list = []
    knowledge_list = []
    population_list = []
    numerosity_list = []
    reliable_list = []
    
    for i in range(n):
        
        # Initialize environment
        maze = gym.make(env.id)
        
        # Reset it, by putting an agent into random position
        situation = maze.reset()
    
        # Training of BACS - Exploration
        agent_explore = BACS(cfg_explore)
        population_explore, metrics_explore = agent_explore.explore(maze, NUMBER_OF_EXPLORE_TRIALS)
    
        # Using BACS - Exploitation
        agent_exploit = BACS(cfg_exploit, population_explore)
        agent_exploit.clean_population()
        population_exploit, metrics_exploit = agent_exploit.exploit(maze, NUMBER_OF_EXPLOIT_TRIALS)

        # Plotting performance
        #metrics_df = parse_metrics_to_df(metrics_explore, metrics_exploit)
        #plot_performance(agent_exploit, maze, metrics_df, cfg_exploit, env.id)
        
        # Get average in exploration
        avg_step_explore = 0
        for trial in metrics_explore:
            avg_step_explore += trial['steps_in_trial']
        avg_step_explore /= NUMBER_OF_EXPLORE_TRIALS
        avg_explore_list.append(avg_step_explore)
        
        # Get average in exploitation
        avg_step_exploit = 0
        for trial in metrics_exploit:
            avg_step_exploit += trial['steps_in_trial']
        avg_step_exploit /= NUMBER_OF_EXPLOIT_TRIALS
        avg_exploit_list.append(avg_step_exploit)
        
        # Get other metrics
        knowledge_list.append(metrics_exploit[-1]['knowledge'])
        population_list.append(metrics_exploit[-1]['population'])
        numerosity_list.append(metrics_exploit[-1]['numerosity'])
        reliable_list.append(metrics_exploit[-1]['reliable'])
        
        print(env.id,'|','#' * (i+1) ,' ' * (n-(i+1)), '|')
    
    # Compute the means and standard deviations
    avg_explore = float(sum(avg_explore_list)) / n
    std_explore = statistics.pstdev(avg_explore_list)
    avg_exploit = float(sum(avg_exploit_list)) / n
    std_exploit = statistics.pstdev(avg_exploit_list)
    avg_knowledge = float(sum(knowledge_list)) / n
    std_knowledge = statistics.pstdev(knowledge_list)
    avg_population = float(sum(population_list)) / n
    std_population = statistics.pstdev(population_list)
    avg_numerosity = float(sum(numerosity_list)) / n
    std_numerosity = statistics.pstdev(numerosity_list)
    avg_reliable = float(sum(reliable_list)) / n
    std_reliable = statistics.pstdev(reliable_list)
    
    result = {
        'maze'             : env.id,
        'avg_explore'      : avg_explore,
        'std_explore'      : std_explore,
        'avg_exploit'      : avg_exploit,
        'std_exploit'      : std_exploit,
        'avg_knowledge'    : avg_knowledge,
        'std_knowledge'    : std_knowledge,
        'avg_population'   : avg_population,
        'std_population'   : std_population,
        'avg_numerosity'   : avg_numerosity,
        'std_numerosity'   : std_numerosity,
        'avg_reliable'     : avg_reliable,
        'std_reliable'     : std_reliable,
        'avg_exploit_list' : avg_exploit_list
    }
    
    print(result)
    
    return result

Custom functions for getting available environments in Gym depending on the type of mazes :

In [5]:
filter_envs_typeIII = lambda env: env.id.startswith("Maze10-") or env.id.startswith("MazeE") \
    or env.id.startswith("Woods10")

filter_envs_typeII = lambda env: env.id.startswith("MazeF4") or env.id.startswith("Maze7") \
    or env.id.startswith("Sutton") or env.id.startswith("MiyazakiB") \
    or env.id.startswith("Lab1")

filter_envs_typeI = lambda env: env.id.startswith("MazeB") or env.id.startswith("MazeD") \
    or env.id.startswith("Littman") or env.id.startswith("MiyazakiA") \
    or env.id.startswith("Cassandra")

filter_envs_na = lambda env: env.id.startswith("MazeF1") or env.id.startswith("MazeF2") \
    or env.id.startswith("MazeF3") or env.id.startswith("Woods1-") \
    or env.id.startswith("Woods14") or env.id.startswith("Maze4") \
    or env.id.startswith("Maze5") or env.id.startswith("MazeA")

all_envs = [env for env in gym.envs.registry.all()]

Benchmarking without the genetic algorithms :

In [ ]:
#maze_envs = [env for env in all_envs if filter_envs_na(env) or filter_envs_typeI(env) or filter_envs_typeII(env) or filter_envs_typeIII(env)]
maze_envs = [env for env in all_envs if filter_envs_typeIII(env)]

results = parmap.map(bench_on_maze, maze_envs, NUMBER_OF_ITERATIONS_TO_BENCH, pm_pbar=True, pm_processes=8)

  0%|          | 0/7 [00:00<?, ?it/s]

Woods100-v0 | #                               |
Woods100-v0 | ##                              |
Woods100-v0 | ###                             |
Woods100-v0 | ####                            |
Woods100-v0 | #####                           |
Woods100-v0 | ######                          |
Woods101-v0 | #                               |
Woods100-v0 | #######                         |
Woods100-v0 | ########                        |
Woods100-v0 | #########                       |
Woods100-v0 | ##########                      |
Woods100-v0 | ###########                     |
Woods101demi-v0 | #                               |
Woods100-v0 | ############                    |
Woods100-v0 | #############                   |
Woods101-v0 | ##                              |
Woods102-v0 | #                               |
Woods100-v0 | ##############                  |
Woods100-v0 | ###############                 |
Woods100-v0 | ################                |
Woods100-v0 | #################     

 14%|█▍        | 1/7 [03:10<19:00, 190.01s/it]

Woods101-v0 | #####                           |
Maze10-v0 | ##                              |
Woods101demi-v0 | ###                             |
Woods101-v0 | ######                          |
Woods102-v0 | ###                             |
Woods101-v0 | #######                         |
Woods101demi-v0 | ####                            |
Maze10-v0 | ###                             |
Woods101-v0 | ########                        |
Woods102-v0 | ####                            |
Woods101demi-v0 | #####                           |
Woods101-v0 | #########                       |
MazeE1-v0 | #                               |
Woods101-v0 | ##########                      |
Woods101demi-v0 | ######                          |
Maze10-v0 | ####                            |
Woods102-v0 | #####                           |
Woods101-v0 | ###########                     |
Woods101demi-v0 | #######                         |
Woods101-v0 | ############                    |
Maze10-v0 | #####           

Parsing the previous result to get a markdown string for each environment :

In [ ]:
markdown_str = ''
for item in results:
    markdown_str += '|' + item['maze'] + '|'
    markdown_str += "{:.2f}".format(item['avg_explore']) + '|'
    markdown_str += "{:.2f}".format(item['std_explore']) + '|'
    markdown_str += "{:.2f}".format(item['avg_exploit']) + '|'
    markdown_str += "{:.2f}".format(item['std_exploit']) + '|'
    markdown_str += "{:.2f}".format(min(item['avg_exploit_list'])) + '|'
    markdown_str += "{:.2f}".format(max(item['avg_exploit_list'])) + '|'
    markdown_str += '0' + '|'
    markdown_str += "{:.2f}".format(item['avg_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['std_knowledge']) + '|'
    markdown_str += "{:.2f}".format(item['avg_population']) + '|'
    markdown_str += "{:.2f}".format(item['std_population']) + '|'
    markdown_str += "{:.2f}".format(item['avg_reliable']) + '|'
    markdown_str += "{:.2f}".format(item['std_reliable']) + '|'
    markdown_str += '\n'
print(markdown_str)

### Last results
 
The parameters used are:
 
- no use of action planning
- use of subsumption in the anticipatory learning process
- use od subsumption in the genetic algorithm is this last one is used
- $\gamma$ = 0.95
- $\theta_i$ = 0.1
- $\theta_r$ = 0.9
- $u_{max}$ = 8 (length of the classifier)
- $\theta_{exp}$ = 20
- $\beta$ = 0.05
- $\epsilon$ = 0.8
- $bs_{max} = 1$
 
If genetic algorithms are used:

- $\theta_{ga}$ = 100
- $\theta_{as}$ = 20
- $\mu$ = 0.3
- $\chi$ = 0.8,
 
The mazes in the following table have been sorted by aliasing type and then by complexity top-down.
 
For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages:
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|30.69|1.86|31.46|23.24|4.69|69.55|0|35.95|1.85|1318.07|95.82|207.70|17.67|
|Woods101demi-v0|29.29|1.36|13.55|6.91|3.04|23.79|0|52.62|0.94|270.17|27.80|80.93|1.88|
|Maze10-v0|48.50|3.15|57.39|9.40|47.98|88.82|0|66.67|2.22|163.80|12.98|48.73|7.58|
|Woods102-v0|29.70|1.53|5.90|3.58|3.18|16.30|0|51.30|0.62|328.23|16.69|123.63|2.95|
|Woods100-v0|12.69|0.45|2.35|0.05|2.23|2.44|0|60.33|1.80|14.40|1.38|10.03|0.18|
|Woods101-v0|38.12|2.25|3.94|0.37|3.59|4.71|0|62.96|0.00|64.83|7.42|43.67|1.51|
|MazeE1-v0|27.36|2.16|4.62|2.21|3.15|12.32|0|42.94|1.80|1494.97|99.63|314.40|26.19|
|**Type II**      ||||||||||||||
|MazeF4-v0|32.63|2.73|4.49|0.08|4.36|4.67|0|84.00|0.00|54.27|11.72|34.57|1.75|
|Maze7-v0|30.36|2.99|4.35|0.07|4.20|4.46|0|80.95|0.00|42.13|3.41|33.73|1.31|
|Sutton-v0|71.87|6.13|50.14|14.10|8.22|71.98|0|48.42|0.71|1912.00|112.79|416.97|19.55|
|Lab1-v0|98.98|0.54|99.89|0.19|99.47|100.00|0|61.76|1.65|672.50|59.83|279.20|12.89|
|MiyazakiB-v0|40.23|5.48|4.11|0.85|3.47|8.37|0|77.67|0.84|566.83|63.03|267.27|13.99|
|**Type I**       ||||||||||||||
|MazeB-v0|39.56|3.23|5.27|0.44|5.00|6.00|0|79.48|1.70|279.70|21.35|180.57|11.57|
|Littman89-v0|25.15|2.68|4.26|0.23|3.99|4.85|0|71.56|0.84|212.30|14.55|133.63|5.41|
|MiyazakiA-v0|25.48|3.05|3.52|0.20|3.22|3.97|0|65.56|2.84|932.43|98.76|302.27|23.15|
|MazeD-v0|37.38|3.40|4.00|0.00|4.00|4.00|0|79.65|3.59|247.27|15.51|176.03|9.02|
|Cassandra4x4-v0|16.06|2.00|2.89|0.26|2.42|3.51|0|51.36|1.09|676.27|63.07|140.33|11.03|
|Littman57-v0|23.05|2.58|16.70|22.89|3.69|64.68|0|72.68|2.98|66.07|5.84|31.90|1.45|
|**Not Aliased**||||||||||||||
|Maze4-v0|32.49|2.63|3.49|0.05|3.40|3.60|0|100.00|0.00|173.03|6.23|173.00|6.19|
|Maze5-v0|47.14|3.04|4.65|0.09|4.52|4.91|0|100.00|0.00|227.70|7.82|227.60|7.93|
|MazeA-v0|63.46|2.59|6.03|0.18|6.00|7.00|0|100.00|0.00|109.83|4.74|109.83|4.74|
|MazeF1-v0|11.78|0.47|1.81|0.03|1.73|1.87|0|100.00|0.00|14.10|0.30|14.10|0.30|
|MazeF2-v0|19.92|0.59|2.50|0.04|2.42|2.58|0|100.00|0.00|18.10|0.30|18.10|0.30|
|MazeF3-v0|29.86|0.92|3.37|0.07|3.20|3.52|0|100.00|0.00|28.87|1.06|28.87|1.06|
|Woods1-v0|9.58|0.45|1.63|0.02|1.59|1.67|0|100.00|0.00|54.27|1.39|54.27|1.39|
|Woods14-v0|70.75|1.33|9.47|0.22|8.97|9.77|0|100.00|0.00|38.50|2.29|38.50|2.29|

 For 1000 trials in Exploration and 500 trials in Exploitation for all environments, computed 30 times for calculating all averages, having $bs_{max} = 2$
 
 
| | Exploration Avg | Exploration Std | Exploration Avg | Exploration Std | Best Exploration | Worst Exploration | Successful tries | Knowledge Avg | Knowledge Std | Population Avg | Population Std | Reliable Avg | Reliable Std |
|------------|:-----:|:-----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
|**Type III**     ||||||||||||||
|MazeE2-v0|30.69|1.86|31.46|23.24|4.69|69.55|0|35.95|1.85|1318.07|95.82|207.70|17.67|
|Woods101demi-v0|29.29|1.36|13.55|6.91|3.04|23.79|0|52.62|0.94|270.17|27.80|80.93|1.88|
|Maze10-v0|48.50|3.15|57.39|9.40|47.98|88.82|0|66.67|2.22|163.80|12.98|48.73|7.58|
|Woods102-v0|29.70|1.53|5.90|3.58|3.18|16.30|0|51.30|0.62|328.23|16.69|123.63|2.95|
|Woods100-v0|12.69|0.45|2.35|0.05|2.23|2.44|0|60.33|1.80|14.40|1.38|10.03|0.18|
|Woods101-v0|38.12|2.25|3.94|0.37|3.59|4.71|0|62.96|0.00|64.83|7.42|43.67|1.51|
|MazeE1-v0|27.36|2.16|4.62|2.21|3.15|12.32|0|42.94|1.80|1494.97|99.63|314.40|26.19|

### Complexity and Optimal Performance by Maze

These measures come from Zatuchna and Bagnall paper :

> Zhanna V. Zatuchna and Anthony Bagnall. 2009. Learning Mazes with Aliasing States: An LCS Algorithm with Associative Perception. Adaptive Behavior - Animals, Animats, Software Agents, Robots, Adaptive Systems 17, 1 (February 2009), 28-57. DOI=http://dx.doi.org/10.1177/1059712308099230

$\phi$ is about the average distance to reward in maze. Its calculation depends mainly on the type of maze. Here is reported $\phi$' if the maze is aliased, the original $\phi$ otherwise.

$\psi$ is about the complexity of the maze that depends on the average distance to reward and on the average steps taken by trained Q-learning.

I also added $d$, the maximal distance to food I consider to count the number of succesfull tries.

The question marks (**?**) highlight some discrepancies present in the original paper.

The asterisks (**\***) indicate pieces of information were not provided.

For further details, please see the original paper.

#### For Type III Aliased Mazes

|       |MazeE2      |Woods101demi|Maze10      |Woods102    |Woods100    |Woods101    |MazeE1       |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:-----------:|
|$d$    |3           |4           |8           |4           |3           |4           |4            |
|$\phi$'|2.33        |3.1         |5.17        |3.31        |2.33        |2.9         |3.07         |
|$\psi$ |251.2       |251 **?**   |171         |167         |166         |149         |167 **?**    |

#### For Type II Aliased Mazes

|       |MazeF4      |Maze7       |Sutton      |Lab1        |MiyazakiB   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |7           |7           | 10         | **\***     |5           |
|$\phi$'|4.5 **?**   |4.33        | **\***     | **\***     |3.33        |
|$\psi$ |47 **?**    |82 **?**    | **?**      | **?**      |1.03        |

#### For Type I Aliased Mazes

|       |MazeB       |Littman89   |MiyazakiA   |MazeD       |Cassandra4x4|Littman57   |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |5           |7           |5           |4           |3           |8           |
|$\phi$'|3.5         |3.77        |3.05        |2.75        |2.27        |3.71        |
|$\psi$ |1.26        |61 **?**    |69 **?**    |1.03        |1           |154 **?**   |

#### For Non Aliased Mazes

|       |Maze4       |Maze5       |MazeA       |MazeF1      |MazeF2      |MazeF3      |Woods1      |Woods14     |
|-------|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|:----------:|
|$d$    |5           |8           |6           |3           |4           |5           |2           |18          |
|$\phi$ |3.5         |4.61        |4.23        |1.8         |2.5         |3.38        |1.63        |9.5         |
|$\psi$ | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     | **\***     |